In [ ]:
!pip -q install wandb

import os, math, time, contextlib, copy
from dataclasses import dataclass, asdict
from typing import Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import amp
from tqdm import trange

import wandb
wandb.login()

# ---- Speed knobs ----
def enable_tf32():
    if torch.cuda.is_available():
        if hasattr(torch.backends.cuda.matmul, "fp32_precision"):
            torch.backends.cuda.matmul.fp32_precision = "tf32"
        else:
            torch.backends.cuda.matmul.allow_tf32 = True

        if (
            hasattr(torch.backends, "cudnn")
            and hasattr(torch.backends.cudnn, "conv")
            and hasattr(torch.backends.cudnn.conv, "fp32_precision")
        ):
            torch.backends.cudnn.conv.fp32_precision = "tf32"
        else:
            torch.backends.cudnn.allow_tf32 = True

    try:
        torch.set_float32_matmul_precision("high")
    except Exception:
        pass

enable_tf32()

def flash_sdpa_context():
    """Request FlashAttention kernels when available; safe fallback otherwise."""
    try:
        from torch.nn.attention import sdpa_kernel, SDPBackend
        return sdpa_kernel([SDPBackend.FLASH_ATTENTION])
    except Exception:
        return contextlib.nullcontext()


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nathan-henry (nathan-henry-uc-berkeley-electrical-engineering-computer) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


In [ ]:
# -------------------------
# Model components
# -------------------------
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.fc_1 = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.gelu = nn.GELU(approximate="tanh")
        self.fc_2 = nn.Linear(3 * config.n_embd, config.n_embd)

    def forward(self, x):
        return self.fc_2(self.gelu(self.fc_1(x)))


class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_heads == 0
        self.n_embd = config.n_embd
        self.n_heads = config.n_heads
        self.head_dim = config.n_embd // config.n_heads

        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = True

    def forward(self, x):
        B, T, C = x.size()

        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)

        q = q.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)

        y = F.scaled_dot_product_attention(q, k, v, dropout_p=0.0, is_causal=True)
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.c_proj(y)
        return y


class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.attn = CausalSelfAttention(config)
        self.mlp = MLP(config)
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.ln_2 = nn.LayerNorm(config.n_embd)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.n_layers = config.n_layers

        self.transformer = nn.ModuleDict(
            dict(
                wte=nn.Embedding(config.vocab_size, config.n_embd),
                wpe=nn.Embedding(config.max_seq_len, config.n_embd),
                h=nn.ModuleList([Block(config) for _ in range(config.n_layers)]),
                ln_f=nn.LayerNorm(config.n_embd),
            )
        )
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.lm_head.weight = self.transformer.wte.weight  # weight tying
        self.apply(self._init_weights)

        self.register_buffer("pos_idx", torch.arange(config.max_seq_len), persistent=False)

    def _init_weights(self, module):
        std = 0.02
        if isinstance(module, nn.Linear):
            if hasattr(module, "NANOGPT_SCALE_INIT"):
                std *= (2 * self.n_layers) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)

    def forward(self, idx, return_full_logits: bool = False):
        B, T = idx.size()
        assert T <= self.config.max_seq_len, f"T={T} exceeds max_seq_len={self.config.max_seq_len}"

        pos = self.transformer.wpe(self.pos_idx[:T])
        x = self.transformer.wte(idx) if self.config.without_pos else (self.transformer.wte(idx) + pos)

        for block in self.transformer.h:
            x = block(x)

        x = self.transformer.ln_f(x)

        # targets are the sorted half (tokens after SEP)
        targets = idx[:, self.config.block_size + 1 :]  # (B, block_size)

        if return_full_logits:
            logits = self.lm_head(x)
            logits_for_loss = logits[:, self.config.block_size : T - 1, :]
        else:
            x_for_loss = x[:, self.config.block_size : T - 1, :]
            logits_for_loss = self.lm_head(x_for_loss)
            logits = logits_for_loss

        loss = F.cross_entropy(
            logits_for_loss.reshape(-1, logits_for_loss.size(-1)),
            targets.reshape(-1),
        )
        return logits, loss


class GPTConfig:
    def __init__(
        self,
        block_size: int,
        vocab_size: int,
        n_layers: int = 2,
        n_heads: int = 1,
        n_embd: int = 64,
        without_pos: bool = False,
    ):
        self.block_size = int(block_size)
        self.vocab_size = int(vocab_size)
        self.n_layers = int(n_layers)
        self.n_heads = int(n_heads)
        self.n_embd = int(n_embd)
        self.without_pos = bool(without_pos)
        self.max_seq_len = 2 * self.block_size + 1


# -------------------------
# Data batching
# -------------------------
SEP_TOKEN = "SEP"

def _sample_numbers(batch_size: int, vocab_n: int, block_size: int, device: torch.device, allow_duplicates: bool):
    if allow_duplicates:
        return torch.randint(0, vocab_n, (batch_size, block_size), device=device, dtype=torch.long)

    if block_size > vocab_n:
        raise ValueError(f"allow_duplicates=False requires block_size <= vocab_n (got block_size={block_size}, vocab_n={vocab_n})")

    # Uniform random subset + random order via random scores + topk
    scores = torch.rand(batch_size, vocab_n, device=device)
    return scores.topk(block_size, dim=1).indices.to(torch.long)

def get_batch(batch_size: int, device: torch.device, vocab_n: int, block_size: int, allow_duplicates: bool):
    """
    Returns idx: (B, 2*block_size + 1)
      [x (len=block_size), SEP, sort(x)]
    Vocabulary:
      numbers: 0..vocab_n-1
      SEP id : vocab_n   (total vocab = vocab_n + 1)
    """
    x = _sample_numbers(batch_size, vocab_n, block_size, device, allow_duplicates)
    vals = x.sort(dim=1).values
    sep_id = vocab_n
    sep = torch.full((batch_size, 1), sep_id, device=device, dtype=torch.long)
    return torch.cat([x, sep, vals], dim=1)


# -------------------------
# Optim + schedule + plateau helpers
# -------------------------
def create_optimizer(model, weight_decay: float, lr: float):
    params = [p for p in model.parameters() if p.requires_grad]
    decay_params = [p for p in params if p.dim() > 1]
    nondecay_params = [p for p in params if p.dim() <= 1]
    optim_groups = [
        {"params": decay_params, "weight_decay": weight_decay},
        {"params": nondecay_params, "weight_decay": 0.0},
    ]
    try:
        return torch.optim.AdamW(optim_groups, lr=lr, betas=(0.9, 0.95), eps=1e-8, fused=True)
    except TypeError:
        return torch.optim.AdamW(optim_groups, lr=lr, betas=(0.9, 0.95), eps=1e-8)

@dataclass
class TrainConfig:
    # Task/data
    vocab_n: int = 1024
    block_size: int = 32
    allow_duplicates: bool = True
    sep_token: str = SEP_TOKEN

    # Model
    n_layers: int = 2
    n_heads: int = 1
    n_embd: int = 64
    without_pos: bool = False

    # Optim
    warmup_iters: int = 200
    max_iters: int = 120000
    learning_rate: float = 1e-4
    min_lr: float = 1e-6
    weight_decay: float = 0.0

    # Batch
    micro_batch_size: int = 1024
    effective_batch_size: int = 4096

    # Logging / ckpt
    log_interval: int = 250
    ckpt_interval: int = 20000
    save_dir: str = "./saved_models"

    # Plateau early stop (log–log)
    # We fit slope of y=log10(loss) vs x=log10(step) over the last `plateau_window_logs` LOG POINTS.
    # Plateau triggers when:
    #   - improvement in log10(loss) across the window is tiny, AND
    #   - the fitted slope is close to 0 (not very negative),
    # for `plateau_patience_logs` consecutive checks.
    plateau_window_logs: int = 40              # window length in *logged points* (each point spaced by log_interval steps)
    plateau_slope_threshold: float = 0.02      # plateau if slope > -this (e.g. -0.01 is "flat-ish")
    plateau_log10_loss_delta: float = 0.02     # plateau if log10(loss) improves by < this across the window (~<4.7% improvement)
    plateau_patience_logs: int = 2             # require plateau condition for this many consecutive checks
    plateau_extra_logs: int = 1                # once plateau confirmed, run this many extra log intervals and then stop
    plateau_min_iters: int = 20000             # don't even check plateau until this many optimizer steps completed

    # Misc
    seed: int = 1337
    use_compile: bool = False

    # W&B
    wandb_project: str = "sortgpt"
    wandb_entity: Optional[str] = None
    wandb_group: Optional[str] = None
    wandb_mode: Optional[str] = None  # "online" | "offline" | "disabled"

def get_lr(itr: int, cfg: TrainConfig) -> float:
    if itr < cfg.warmup_iters:
        return cfg.learning_rate * (itr + 1) / (cfg.warmup_iters + 1)
    if itr > cfg.max_iters:
        return cfg.min_lr
    ratio = (itr - cfg.warmup_iters) / (cfg.max_iters - cfg.warmup_iters)
    ratio = 0.5 * (1.0 + math.cos(math.pi * ratio))
    return cfg.min_lr + ratio * (cfg.learning_rate - cfg.min_lr)

def _safe_log10(x: float, eps: float = 1e-30) -> float:
    return math.log10(max(float(x), eps))

def _linreg_slope(xs, ys) -> float:
    """Slope of y vs x via least squares."""
    n = len(xs)
    if n < 2:
        return 0.0
    x_mean = sum(xs) / n
    y_mean = sum(ys) / n
    cov = 0.0
    var = 0.0
    for x, y in zip(xs, ys):
        dx = x - x_mean
        dy = y - y_mean
        cov += dx * dy
        var += dx * dx
    return cov / var if var > 0 else 0.0

def make_wandb_run_name(cfg: TrainConfig) -> str:
    return f"sortgpt_N{cfg.vocab_n}_K{cfg.block_size}_L{cfg.n_layers}_dup{int(cfg.allow_duplicates)}"

def make_save_filename(prefix: str, cfg: TrainConfig, iters_done: int) -> str:
    # prefix in {"Final", "Checkpoint"}
    return f"{prefix}_N{cfg.vocab_n}_L{cfg.n_layers}_dup{int(cfg.allow_duplicates)}_iters{int(iters_done)}.pt"

@torch.no_grad()
def eval_train_loss_once(model: nn.Module, device: torch.device, cfg: TrainConfig, use_amp: bool, amp_dtype: Optional[torch.dtype]):
    model.eval()
    batch = get_batch(
        batch_size=cfg.micro_batch_size,
        device=device,
        vocab_n=cfg.vocab_n,
        block_size=cfg.block_size,
        allow_duplicates=cfg.allow_duplicates,
    )
    if use_amp:
        with amp.autocast(device_type="cuda", dtype=amp_dtype):
            _, loss = model(batch, return_full_logits=False)
    else:
        _, loss = model(batch, return_full_logits=False)
    model.train()
    return float(loss.item())

def train_sorting_gpt(cfg: TrainConfig):
    # ---- seeds ----
    torch.manual_seed(cfg.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(cfg.seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"using device: {device}")

    # No-duplicate sanity checks
    if not cfg.allow_duplicates:
        if cfg.block_size > cfg.vocab_n:
            raise ValueError("allow_duplicates=False requires block_size <= vocab_n")
        if cfg.block_size == cfg.vocab_n:
            print("⚠️  Warning: block_size == vocab_n with allow_duplicates=False => sorted output is constant (degenerate task).")

    total_vocab_size = cfg.vocab_n + 1
    sep_id = cfg.vocab_n
    max_seq_len = 2 * cfg.block_size + 1

    grad_accum_steps = cfg.effective_batch_size // cfg.micro_batch_size
    assert cfg.effective_batch_size % cfg.micro_batch_size == 0, "effective_batch_size must be divisible by micro_batch_size"

    # ---- AMP ----
    use_amp = (device.type == "cuda")
    if use_amp:
        bf16_ok = getattr(torch.cuda, "is_bf16_supported", lambda: False)()
        amp_dtype = torch.bfloat16 if bf16_ok else torch.float16
    else:
        amp_dtype = None

    scaler = amp.GradScaler(enabled=(use_amp and amp_dtype == torch.float16))

    # ---- model ----
    model_cfg = GPTConfig(
        block_size=cfg.block_size,
        vocab_size=total_vocab_size,
        n_layers=cfg.n_layers,
        n_heads=cfg.n_heads,
        n_embd=cfg.n_embd,
        without_pos=cfg.without_pos,
    )
    model = GPT(model_cfg).to(device)

    if cfg.use_compile and hasattr(torch, "compile"):
        try:
            model = torch.compile(model, mode="max-autotune")
            print("torch.compile enabled")
        except Exception as e:
            print(f"torch.compile failed, continuing uncompiled: {e}")

    optimizer = create_optimizer(model, weight_decay=cfg.weight_decay, lr=cfg.learning_rate)
    os.makedirs(cfg.save_dir, exist_ok=True)

    # ---- W&B init ----
    if wandb.run is not None:
        wandb.finish()

    wandb_cfg = asdict(cfg)
    wandb_cfg.update(
        dict(
            total_vocab_size=total_vocab_size,
            sep_id=sep_id,
            max_seq_len=max_seq_len,
            grad_accum_steps=grad_accum_steps,
            amp_dtype=str(amp_dtype) if amp_dtype is not None else "none",
            device=str(device),
        )
    )

    run = wandb.init(
        project=cfg.wandb_project,
        entity=cfg.wandb_entity,
        group=cfg.wandb_group,
        name=make_wandb_run_name(cfg),
        config=wandb_cfg,
        mode=cfg.wandb_mode,
    )

    # Raw metrics
    run.define_metric("iter")
    run.define_metric("train/*", step_metric="iter")
    run.define_metric("lr", step_metric="iter")
    run.define_metric("plateau/*", step_metric="iter")

    # Log–log coordinates metrics (these charts are “log–log” without UI toggles)
    run.define_metric("ll/iter")
    run.define_metric("ll/*", step_metric="ll/iter")

    # ---- plateau state ----
    ll_iters_hist = []
    ll_loss_hist = []
    plateau_hits = 0
    plateau_reached = False
    plateau_reached_iter = None
    stop_after_log_iter = None  # iteration index (itr) at which we stop after logging

    last_logged_itr = None
    iters_done = cfg.max_iters
    last_log_t = time.time()

    with flash_sdpa_context():
        for itr in trange(cfg.max_iters, desc="training"):
            optimizer.zero_grad(set_to_none=True)
            loss_accum = torch.zeros((), device=device)

            for _ in range(grad_accum_steps):
                batch = get_batch(
                    batch_size=cfg.micro_batch_size,
                    device=device,
                    vocab_n=cfg.vocab_n,
                    block_size=cfg.block_size,
                    allow_duplicates=cfg.allow_duplicates,
                )

                if use_amp:
                    with amp.autocast(device_type="cuda", dtype=amp_dtype):
                        _, loss = model(batch, return_full_logits=False)
                else:
                    _, loss = model(batch, return_full_logits=False)

                loss_to_back = loss / grad_accum_steps
                if scaler.is_enabled():
                    scaler.scale(loss_to_back).backward()
                else:
                    loss_to_back.backward()

                loss_accum = loss_accum + loss.detach()

            lr = get_lr(itr, cfg)
            for pg in optimizer.param_groups:
                pg["lr"] = lr

            if scaler.is_enabled():
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()

            # ---- log + plateau check ----
            if itr % cfg.log_interval == 0:
                train_loss_t = (loss_accum / grad_accum_steps).detach()
                train_loss = float(train_loss_t.item())  # sync only here

                steps_done = itr + 1
                ll_iter = _safe_log10(steps_done)
                ll_loss = _safe_log10(train_loss)
                ll_lr = _safe_log10(lr)

                ll_iters_hist.append(ll_iter)
                ll_loss_hist.append(ll_loss)

                slope = None
                delta_ll_loss = None
                plateau_now = 0

                # Only attempt plateau detection when we have enough log points and have trained long enough
                if (
                    (not plateau_reached)
                    and (len(ll_iters_hist) >= cfg.plateau_window_logs)
                    and (steps_done >= cfg.plateau_min_iters)
                ):
                    xs = ll_iters_hist[-cfg.plateau_window_logs:]
                    ys = ll_loss_hist[-cfg.plateau_window_logs:]

                    slope = _linreg_slope(xs, ys)
                    delta_ll_loss = ys[0] - ys[-1]  # improvement in log10(loss); should be >=0 when improving

                    is_improving = (delta_ll_loss >= 0.0)
                    plateau_now = int(
                        is_improving
                        and (delta_ll_loss < cfg.plateau_log10_loss_delta)
                        and (slope > -cfg.plateau_slope_threshold)
                    )

                    plateau_hits = (plateau_hits + 1) if plateau_now else 0

                    if plateau_hits >= cfg.plateau_patience_logs:
                        plateau_reached = True
                        plateau_reached_iter = itr
                        stop_after_log_iter = itr + cfg.log_interval * max(int(cfg.plateau_extra_logs), 0)
                        print(
                            f"🟨 plateau detected @ itr={itr} "
                            f"(slope={slope:.4f}, Δlog10(loss)={delta_ll_loss:.4f}); "
                            f"will stop after logging itr={stop_after_log_iter}"
                        )

                now = time.time()
                dt = now - last_log_t
                last_log_t = now
                print(f"itr: {itr} lr: {lr:.3e} train loss: {train_loss:.6f} (dt={dt:.2f}s)")

                log_dict = {
                    "iter": steps_done,
                    "train/loss": train_loss,
                    "lr": lr,

                    # log–log coordinates stream
                    "ll/iter": ll_iter,
                    "ll/train_loss": ll_loss,
                    "ll/lr": ll_lr,

                    # plateau telemetry
                    "plateau/now": plateau_now,
                    "plateau/hits": plateau_hits,
                    "plateau/reached": int(plateau_reached),
                }
                if slope is not None:
                    log_dict["plateau/loglog_slope"] = float(slope)
                if delta_ll_loss is not None:
                    log_dict["plateau/delta_log10_loss_window"] = float(delta_ll_loss)

                run.log(log_dict, step=steps_done)
                last_logged_itr = itr

                # Stop after extra log(s) once plateau is confirmed
                if plateau_reached and (stop_after_log_iter is not None) and (itr >= stop_after_log_iter):
                    iters_done = steps_done
                    break

            # ---- checkpointing ----
            if cfg.ckpt_interval and (itr > 0) and (itr % cfg.ckpt_interval == 0):
                steps_done = itr + 1
                ckpt_name = make_save_filename("Checkpoint", cfg, steps_done)
                ckpt_path = os.path.join(cfg.save_dir, ckpt_name)
                torch.save(
                    {
                        "itr": itr,
                        "iters_done": steps_done,
                        "plateau_reached": plateau_reached,
                        "plateau_reached_iter": plateau_reached_iter,
                        "train_config": wandb_cfg,
                        "model_config": vars(model_cfg),
                        "model": model.state_dict(),
                        "optimizer": optimizer.state_dict(),
                    },
                    ckpt_path,
                )
                print(f"saved checkpoint: {ckpt_path}")

    # If we didn't log the very last step, log one final point
    final_itr = iters_done - 1
    if last_logged_itr is None or last_logged_itr != final_itr:
        final_loss = eval_train_loss_once(model, device, cfg, use_amp, amp_dtype)
        final_lr = get_lr(final_itr, cfg)
        steps_done = iters_done

        run.log(
            {
                "iter": steps_done,
                "train/loss": final_loss,
                "lr": final_lr,
                "ll/iter": _safe_log10(steps_done),
                "ll/train_loss": _safe_log10(final_loss),
                "ll/lr": _safe_log10(final_lr),
                "plateau/reached": int(plateau_reached),
                "plateau/hits": plateau_hits,
                "plateau/now": 0,
            },
            step=steps_done,
        )

    # Final save (iters_done computed after run)
    final_name = make_save_filename("Final", cfg, iters_done)
    final_path = os.path.join(cfg.save_dir, final_name)
    torch.save(
        {
            "iters_done": iters_done,
            "plateau_reached": plateau_reached,
            "plateau_reached_iter": plateau_reached_iter,
            "train_config": wandb_cfg,
            "model_config": vars(model_cfg),
            "model": model.state_dict(),
        },
        final_path,
    )
    print(f"saved final model: {final_path}")

    run.summary["iters_done"] = iters_done
    run.summary["plateau_reached"] = bool(plateau_reached)
    run.summary["plateau_reached_iter"] = int(plateau_reached_iter) if plateau_reached_iter is not None else None
    run.finish()

    return final_path


# -------------------------
# Grid runner (expensive-first, no duplicates only)
# -------------------------
def run_grid(base_cfg: TrainConfig, vocab_sizes, layer_counts):
    # expensive first: vocab desc, then layers desc
    vocab_sizes = sorted(list(vocab_sizes), reverse=True)
    layer_counts = sorted(list(layer_counts), reverse=True)

    for N in vocab_sizes:
        for L in layer_counts:
            cfg = copy.deepcopy(base_cfg)
            cfg.vocab_n = int(N)
            cfg.n_layers = int(L)
            cfg.allow_duplicates = False  # per your request (no dup grid)
            train_sorting_gpt(cfg)


In [ ]:
import os, time

STAMP = time.strftime("%Y%m%d_%H%M%S")

# Keep local wandb run files isolated (doesn't affect online grouping)
os.environ["WANDB_DIR"] = f"./wandb_grid_bs8192_{STAMP}"
SAVE_DIR = f"./saved_models_grid_bs8192_{STAMP}"

# Use the same project as earlier if you defined PROJECT; otherwise default
PROJECT = globals().get("PROJECT", "sortgpt")
GROUP = f"grid_bs8192_nodup_v256to32_{STAMP}"

print("WANDB_DIR =", os.environ["WANDB_DIR"])
print("SAVE_DIR  =", SAVE_DIR)
print("PROJECT   =", PROJECT)
print("GROUP     =", GROUP)

# --- Override run naming so W&B run titles include the batch size (8192) ---
# train_sorting_gpt() calls make_wandb_run_name(cfg), so redefining it here affects naming.
def make_wandb_run_name(cfg):
    ga = int(cfg.effective_batch_size) // int(cfg.micro_batch_size)
    return (
        f"bs{int(cfg.micro_batch_size)}"
        f"_eb{int(cfg.effective_batch_size)}"
        f"_ga{ga}"
        f"_N{int(cfg.vocab_n)}"
        f"_K{int(cfg.block_size)}"
        f"_L{int(cfg.n_layers)}"
        f"_dup{int(cfg.allow_duplicates)}"
    )

# IMPORTANT for no-duplicates: block_size must be <= smallest vocab size (32),
# and should be < 32 to avoid degenerate constant target when vocab_n=32.
base_cfg = TrainConfig(
    # Task (vocab_n and n_layers will be overridden in run_grid)
    vocab_n=256,
    block_size=31,
    allow_duplicates=False,

    # Model
    n_layers=2,
    n_heads=1,
    n_embd=64,
    without_pos=False,

    # Train
    max_iters=120000,
    warmup_iters=200,
    learning_rate=1e-4,
    min_lr=1e-6,
    weight_decay=0.0,

    # Batch (NO grad accumulation)
    micro_batch_size=8192,
    effective_batch_size=8192,

    # Logging / ckpt
    log_interval=250,
    ckpt_interval=20000,
    save_dir=SAVE_DIR,

    # Plateau stop
    plateau_window_logs=40,
    plateau_min_iters=20000,
    plateau_slope_threshold=0.02,
    plateau_log10_loss_delta=0.02,
    plateau_patience_logs=2,
    plateau_extra_logs=1,

    # W&B
    wandb_project=PROJECT,
    wandb_group=GROUP,
)

run_grid(
    base_cfg,
    vocab_sizes=[256, 128, 64, 32],
    layer_counts=[1, 2, 3],
)


WANDB_DIR = ./wandb_grid_bs8192_20260103_075004
SAVE_DIR  = ./saved_models_grid_bs8192_20260103_075004
PROJECT   = sortgpt
GROUP     = grid_bs8192_nodup_v256to32_20260103_075004
using device: cuda


iter,▁▂▃▄▅▅▆▇█
ll/iter,▁▆▇▇▇████
ll/lr,▁████████
ll/train_loss,██▇▇▆▅▃▂▁
lr,▁████████
plateau/hits,▁▁▁▁▁▁▁▁▁
plateau/now,▁▁▁▁▁▁▁▁▁
plateau/reached,▁▁▁▁▁▁▁▁▁
train/loss,█▇▅▄▃▂▁▁▁
iter,2001
ll/iter,3.30125


training:   0%|          | 6/120000 [00:00<48:07, 41.55it/s]

itr: 0 lr: 4.975e-07 train loss: 5.565396 (dt=0.06s)


training:   0%|          | 257/120000 [00:11<1:25:09, 23.43it/s]

...

training: 100%|█████████▉| 119764/120000 [36:25<00:04, 57.16it/s]

itr: 119750 lr: 1.001e-06 train loss: 0.000000 (dt=4.56s)


training: 100%|██████████| 120000/120000 [36:30<00:00, 54.79it/s]


saved final model: ./saved_models_grid_bs8192_20260103_075004/Final_N32_L1_dup0_iters120000.pt


iter,▁▁▁▁▁▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇██
ll/iter,▁▂▃▃▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████
ll/lr,█████████████▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▃▃▃▁
ll/train_loss,█▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▃▁▁▁▁▁▁▁▁▁▁
lr,████▇▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
plateau/delta_log10_loss_window,▂▂▂▂▂▂▂▂▃▂▂▂▂▃▁▂▂▃▃▂▂▆▂▅▇▇▇█▅▇▃▃▄▃▂▃▂▂▂▂
plateau/hits,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁
plateau/loglog_slope,████████████▇█▇▇████▇▇▇▇▇███▇█▇▁▂▃▇▆▇▇██
plateau/now,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
plateau/reached,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
+1,...
